# Example use of the Sonic module Linckii.jl

In [ ]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

In [ ]:
import Dates, JuliaDB, Plots, Random

In [ ]:
include("../src/Linckii.jl")
include("../src/LinckiiSecrets.jl")

Get access token,

In [ ]:
access = Linckii.get_access(LinckiiSecrets.keab)

GET "https://canary.noda.se/$site_name/api/v1/node"

In [ ]:
@time nodes = Sonic.Linckii.get_nodes(access)

GET "https://canary.noda.se/$site_name/api/v1/device"

In [ ]:
@time devices = Linckii.get_devices(access)

GET "https://canary.noda.se/$site_name/api/v1/sensor"

In [ ]:
@time sensors = Linckii.get_sensors(access)

Save site,

In [ ]:
Linckii.savesite(access; nodes = nodes, devices = devices, sensors = sensors)

Load site,

In [ ]:
nodes, devices, sensors = Linckii.loadsite(access, :nodes, :devices, :sensors)

Get side 1 substation series,

In [ ]:
signals = JuliaDB.reindex(
    JuliaDB.join(
        Linckii.flatten_nodes(JuliaDB.filter(r -> r.device_id == 4, nodes)),
        JuliaDB.filter(
            r -> r.sensor_name in [
                :meter_effect,
                :meter_primreturntemp,
                :meter_primsupplytemp,
                :meter_volumeflow,
            ],
            sensors
        );
        lkey = :sensor_id,
        rkey = :sensor_id,
    ),
    (:node_id, :sensor_id),
)

In [ ]:
signal_rows = JuliaDB.rows(signals)
length(signal_rows)

Save data,

In [ ]:
dates = [Dates.DateTime(yyyy, mm) for yyyy in 2016 : 2019 for mm in 1 : 12]

In [ ]:
if true
    signal_rows = [signal_rows[Random.rand(1 : end)]]
end
for r in signal_rows
    try
        Linckii.savedata(access, r.node_id, r.sensor_name, dates...)
        println("$(Linckii.datapath(access, r.node_id, r.sensor_name)): Done")
    catch e
        println("$(Linckii.datapath(access, r.node_id, r.sensor_name)): $(e)")
    end
end

Load data,

In [ ]:
r = signal_rows[Random.rand(1 : end)]
(r.node_id, r.sensor_name)

In [ ]:
data = Linckii.loaddata(access, r.node_id, r.sensor_name)

Simple plot,

In [ ]:
t = JuliaDB.filter(
    r -> r.datetime >= Dates.DateTime(2018) && r.datetime <= Dates.DateTime(2019),
    data,
)
x = JuliaDB.select(t, :datetime)
y = JuliaDB.select(t, :value)
Plots.plot(x, y, xlabel = :datetime, label = Linckii.datapath(access, r.node_id, r.sensor_name))